# Service Creation
## Load Config

In [2]:
import json
import os

In [43]:
def load_app_config(config_file):
    config_path = os.path.join(config_file)
    if not os.path.exists(config_path):
        raise FileNotFoundError(f"Config file not found: {config_path}")
    with open(config_path, "r") as f:
        return json.load(f)

cfg = load_app_config("config_dev.json")

## Resource Group

In [5]:
from azure.identity import DefaultAzureCredential
from azure.mgmt.resource import ResourceManagementClient

In [6]:
rg_config = cfg.get("resource_group", {})
subscription_id = cfg.get("subscription_id", {})
rg_name = rg_config.get("name")
location = rg_config.get("location", "eastus")

In [7]:
def get_resource_client(subscription_id):
    """Authenticate and return the ResourceManagementClient"""
    credential = DefaultAzureCredential()
    return ResourceManagementClient(credential, subscription_id)

In [8]:
resource_client = get_resource_client(subscription_id)

In [9]:
def validate_resource_group(resource_client, rg_name):
    """Check if a resource group exists"""
    print(f"🔍 Validating resource group '{rg_name}'...")
    exists = resource_client.resource_groups.check_existence(rg_name)
    if exists:
        print(f"✅ Resource group '{rg_name}' exists.")
    else:
        print(f"❌ Resource group '{rg_name}' does not exist.")
    return exists

### Validating if resource group is available if not create one

In [10]:
if validate_resource_group(resource_client, rg_name):
    print(f"ℹ️ Resource group '{rg_name}' already exists.")
else:
    print(f"🚀 Creating resource group '{rg_name}' in '{location}'...")
    rg_result = resource_client.resource_groups.create_or_update(
        rg_name, {"location": location}
    )
    print(f"✅ Resource group '{rg_result.name}' created successfully.")

🔍 Validating resource group 'rg-mymlops'...
❌ Resource group 'rg-mymlops' does not exist.
🚀 Creating resource group 'rg-mymlops' in 'eastus'...
✅ Resource group 'rg-mymlops' created successfully.


## Virtual machine

In [11]:
from azure.identity import DefaultAzureCredential
from azure.mgmt.resource import ResourceManagementClient
from azure.mgmt.compute import ComputeManagementClient
from azure.mgmt.network import NetworkManagementClient
from azure.mgmt.compute.models import (
    HardwareProfile, OSProfile, LinuxConfiguration,
    SshConfiguration, SshPublicKey, NetworkInterfaceReference,
    StorageProfile, ImageReference, OSDisk, DiskCreateOptionTypes
)
from azure.mgmt.network.models import (
    NetworkSecurityGroup, SecurityRule, PublicIPAddress,
    NetworkInterface, Subnet, VirtualNetwork, IPConfiguration
)

In [12]:
subscription_id = cfg["subscription_id"]
resource_group = cfg["resource_group"]["name"]
location = cfg["resource_group"]["location"]
vm_name = cfg["vm"]["name"]
vm_size = cfg["vm"]["size"]
admin_username = cfg["vm"]["admin_username"]
ssh_key_path = os.path.expanduser(cfg["vm"]["ssh_public_key_path"])
virtual_network = cfg["vm"]["virtual_network"]
sub_network = cfg["vm"]["sub_network"]
network_security_grp = cfg["vm"]["network_security_grp"]
public_ip = cfg["vm"]["public_ip"]
network_interface = cfg["vm"]["network_interface"]

In [13]:
print(f"🚀 Starting VM creation: {vm_name} in {location}")

credential = DefaultAzureCredential()
resource_client = ResourceManagementClient(credential, subscription_id)
network_client = NetworkManagementClient(credential, subscription_id)
compute_client = ComputeManagementClient(credential, subscription_id)

🚀 Starting VM creation: vm-mlops in eastus


In [14]:
print(f"🔹 Ensuring resource group '{resource_group}' exists...")
resource_client.resource_groups.create_or_update(
    resource_group, {"location": location})

🔹 Ensuring resource group 'rg-mymlops' exists...


In [15]:
# Virtual Network
print("🔹 Creating Virtual Network...")
vnet = network_client.virtual_networks.begin_create_or_update(
    resource_group, virtual_network,
    {
        "location": location, 
        "address_space": {"address_prefixes": ["10.0.0.0/16"]}
    },
).result()
print(f"✅ Virtual Network '{vnet.name}' created successfully.")

🔹 Creating Virtual Network...
✅ Virtual Network 'vnet-mlops' created successfully.


In [16]:
# Subnet
print("🔹 Creating Subnet...")
subnet = network_client.subnets.begin_create_or_update(
    resource_group, virtual_network, 
    sub_network, {"address_prefix": "10.0.0.0/24"},
).result()
print(f"✅ Subnet '{subnet.name}' created successfully.")

🔹 Creating Subnet...
✅ Subnet 'subnet-mlops' created successfully.


In [17]:
# NSG (SSH only)
print("🔹 Creating NSG (allow SSH)...")
nsg = network_client.network_security_groups.begin_create_or_update(
    resource_group, network_security_grp,
    {
        "location": location,
        "security_rules": [
            SecurityRule(
                name="AllowSSH", protocol="Tcp", direction="Inbound", 
                access="Allow", priority=1000, source_address_prefix="*", 
                destination_address_prefix="*", source_port_range="*", 
                destination_port_range="22",
            )
        ],
    },
).result()
print(f"✅ NSG '{nsg.name}' created successfully.")

🔹 Creating NSG (allow SSH)...
✅ NSG 'nsg-mlops' created successfully.


In [18]:
# Public IP
print("🔹 Creating Public IP...")
public_ip = network_client.public_ip_addresses.begin_create_or_update(
    resource_group, public_ip, 
    {"location": location, "public_ip_allocation_method":"Dynamic"},
    ).result()
print(f"✅ NSG '{public_ip.name}' created successfully.")
# print(f"✅ NSG '{public_ip.ip_address}' created successfully.")

🔹 Creating Public IP...
✅ NSG 'pip-mlops' created successfully.


In [19]:
# Network Interface
print("🔹 Creating Network Interface...")
nic = network_client.network_interfaces.begin_create_or_update(
    resource_group, network_interface,
    {
        "location": location,
        "ip_configurations": [
            IPConfiguration(
                name="ipconfig1",
                subnet=Subnet(id=subnet.id),
                public_ip_address=PublicIPAddress(id=public_ip.id),
                network_security_group=NetworkSecurityGroup(id=nsg.id),
            )
        ],
    },
).result()
print(f"✅ NSG '{nic.name}' created successfully.")

network_security_group is not a known attribute of class <class 'azure.mgmt.network.models._models_py3.IPConfiguration'> and will be ignored


🔹 Creating Network Interface...
✅ NSG 'nic-mlops' created successfully.


In [20]:
# VM Configuration
print("🔹 Defining VM configuration...")
with open(ssh_key_path, "r") as f:
    ssh_key_data = f.read().strip()

vm_parameters = {
    "location": location,
    "hardware_profile": HardwareProfile(vm_size=vm_size),
    "storage_profile": StorageProfile(
        image_reference=ImageReference(
            publisher="Canonical",
            offer="UbuntuServer",
            sku="18.04-LTS",
            version="latest",
        ),
        os_disk=OSDisk(
            create_option=DiskCreateOptionTypes.from_image,
            managed_disk={"storage_account_type": "StandardSSD_LRS"},
        ),
    ),
    "os_profile": OSProfile(
        computer_name=vm_name,
        admin_username=admin_username,
        linux_configuration=LinuxConfiguration(
            disable_password_authentication=True,
            ssh=SshConfiguration(
                public_keys=[
                    SshPublicKey(
                        path=f"/home/{admin_username}/.ssh/authorized_keys",
                        key_data=ssh_key_data,
                    )
                ]
            ),
        ),
    ),
    "network_profile": {
        "network_interfaces": [NetworkInterfaceReference(id=nic.id)]},
}

🔹 Defining VM configuration...


In [21]:
# Create the VM
print(f"🚀 Creating VM '{vm_name}' ... this may take a few minutes.")
creation = compute_client.virtual_machines.begin_create_or_update(
    resource_group, vm_name, vm_parameters
)
vm_result = creation.result()

🚀 Creating VM 'vm-mlops' ... this may take a few minutes.


In [24]:
# Fetch VM details
vm_details = compute_client.virtual_machines.get(resource_group, vm_name)
# Print summary
print("\n🧾 VM Details:")
print(f"Name: {vm_details.name}")
print(f"Location: {vm_details.location}")
print(f"Size: {vm_details.hardware_profile.vm_size}")
print(f"OS Disk: {vm_details.storage_profile.os_disk.name}")
print(f"Image: {vm_details.storage_profile.image_reference.offer} {vm_details.storage_profile.image_reference.sku}")
print(f"Provisioning State: {vm_details.provisioning_state}")


🧾 VM Details:
Name: vm-mlops
Location: eastus
Size: Standard_B1ms
OS Disk: vm-mlops_OsDisk_1_f826069cf2fb491b816883b5d23e0258
Image: UbuntuServer 18.04-LTS
Provisioning State: Succeeded


In [ ]:
# Get NIC attached to the VM
nic_id = vm_details.network_profile.network_interfaces[0].id.split('/')[-1]

# Fetch NIC details
nic = network_client.network_interfaces.get(resource_group, nic_id)

# Check if NSG is attached directly to NIC
if nic.network_security_group:
    nsg_id = nic.network_security_group.id.split('/')[-1]
    nsg = network_client.network_security_groups.get(resource_group, nsg_id)
    print(f"🔒 NSG attached to NIC '{nic.name}': {nsg.name}")
    print(f"Location: {nsg.location}")
    print(f"Security Rules:")
    for rule in nsg.security_rules:
        print(f"  - {rule.name}: {rule.direction} {rule.access} {rule.protocol} {rule.destination_port_range}")
else:
    print(f"No NSG attached directly to NIC '{nic.name}'.")

🔒 NSG attached to NIC 'nic-mlops': nsg-mlops
Location: eastus
Security Rules:
  - AllowSSH: Inbound Allow Tcp 22
No NSG attached to Subnet 'subnet-mlops'.


In [28]:
# Attach NSG to NIC
nic.network_security_group = nsg

In [29]:
# Update NIC with the NSG association
update_nic = network_client.network_interfaces.begin_create_or_update(
    resource_group,
    nic_id,
    nic
)

In [31]:
result = update_nic.result()
print(f"✅ NSG '{nsg.name}' successfully attached to NIC '{nic_id}'.")

✅ NSG 'nsg-mlops' successfully attached to NIC 'nic-mlops'.


In [33]:
if nic.network_security_group:
    nsg_id = nic.network_security_group.id.split('/')[-1]
    nsg = network_client.network_security_groups.get(resource_group, nsg_id)
    print(f"🔒 NSG attached to NIC '{nic.name}': {nsg.name}")
    print(f"Location: {nsg.location}")
    print(f"Security Rules:")
    for rule in nsg.security_rules:
        print(f"  - {rule.name}: {rule.direction} {rule.access} {rule.protocol} {rule.destination_port_range}")
else:
    print(f"No NSG attached directly to NIC '{nic.name}'.")

🔒 NSG attached to NIC 'nic-mlops': nsg-mlops
Location: eastus
Security Rules:
  - AllowSSH: Inbound Allow Tcp 22


## Blob Storage

In [35]:
from azure.mgmt.storage import StorageManagementClient

In [36]:
credential = DefaultAzureCredential()
storage_client = StorageManagementClient(credential, subscription_id)

In [37]:
credential = DefaultAzureCredential()
storage_client = StorageManagementClient(credential, subscription_id)

In [44]:
storage_account_name = cfg["storage"]["storage_account"]
container = cfg["storage"]["containers"]

In [45]:
# Create Blob Storage with hierarchical namespace (Data Lake Gen2)
storage_async_op = storage_client.storage_accounts.begin_create(
    resource_group,
    storage_account_name,
    {
        "location": location,
        "kind": "StorageV2",
        "sku": {"name": "Standard_LRS"},
        "is_hns_enabled": True  # Hierarchical namespace enabled
    },
)
storage_account = storage_async_op.result()
print(f"✅ Storage account '{storage_account.name}' created.")

✅ Storage account 'storagemymlops' created.


In [46]:
# ==== Get Storage Account Key ====
print("🔑 Retrieving access key...")
keys = storage_client.storage_accounts.list_keys(resource_group, storage_account_name)
storage_key = keys.keys[0].value

🔑 Retrieving access key...


'/Users/vinay/Projects/token'

In [58]:
# ==== Save Key to File ====
key_file_path = f"/Users/vinay/Projects/my-mlops/tokens/{storage_account_name}"
print(key_file_path)
with open(key_file_path, "w") as key_file:
    key_file.write(storage_key)
print(f"🗝️ Storage account key saved to: {key_file_path}")

/Users/vinay/Projects/my-mlops/tokens/storagemymlops
🗝️ Storage account key saved to: /Users/vinay/Projects/my-mlops/tokens/storagemymlops
